## importing required libraries

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

# data processing

## function for read and preprocessing images

In [2]:
# Load Haar cascade for face detection this is a model used for identifying faces and giving its coordinates
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
image_size = (224,224)
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) > 0:
        x, y, w, h = faces[0]
        face = image[y:y+h, x:x+w]
        face_resized = cv2.resize(face, image_size)
        # here I am covering the lower side of the image with a black mask so that the model while training focuses on the top features
        face_resized[112:,:] = 0
        return preprocess_input(face_resized)
    else:
        face_resized = cv2.resize(image,image_size)
        face_resized[112:,:] = 0
        return preprocess_input(face_resized)

## loading and processing images and saving them as numpy files

In [4]:
# Set paths and parameters
data_dir = '/Users/saatvik/Desktop/secytasks/CVtask/genderclassifier/training_dataset_uploaded'

# Load dataset and preprocess images
data = []
labels = []
for label in ['Male', 'Female']:
    label_dir = os.path.join(data_dir, label)
    for image_name in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_name)
        preprocessed_image = preprocess_image(image_path)
        if preprocessed_image is not None:
            data.append(preprocessed_image)
            labels.append(0 if label == 'Male' else 1)

data = np.array(data)
labels = np.array(labels)

# saving data so that I wont have to run this cell if the kernal restarts

print("phase 1 done")

phase 1 done


## loading data from saved state
P.S. never load a file of size 11 gb 💀

# building the model As I couldn't download VGG16 I am using colab and will upload tha tin separate .ipynb file

I couldn't run this file on colab due to ram limitations 😭.So, back to running locally and merging both the files

## test train split and using data generators

In [5]:
batch_size = 32
# Split dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

#using data generators
train_datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=10, zoom_range=0.1)
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

# building the model

## model architecture

I am using VGG16 model for the task where I am not including the top layer and adding my own linear layer+sigmoid function to perform logistic regression and training only the final layer

In [11]:
# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# freezing the vgg model so that only final 2 layer gets trained
base_model.trainable = False
# Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# Model to train
model = Model(inputs=base_model.input, outputs=predictions)

## compiling the model

The optimizer I am using is adam optimizer ,loss function - BCEloss and accuracy for metrics

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## training the model

it took around 20 mins to train just the last layer

In [13]:
num_epochs = 1
# Train the model
model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)

285/478 ━━━━━━━━━━━━━━━━━━━━ 10:27 3s/step - accuracy: 0.8081 - loss: 0.4903

KeyboardInterrupt: 

## saving the model

In [16]:
# Save the trained model
model.save('gender_prediction_model.h5')

## loading my model

In [3]:
## loading the model
loaded_model = load_model('gender_prediction_model.h5')
loaded_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,241,027 (58.14 MB)

 Trainable params: 526,337 (2.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2 (12.00 B)

## testing the accuracy of the model

In [14]:
predictions = model.predict(X_val)
predicted_labels = (predictions > 0.5).astype(int).flatten()
accuracy = accuracy_score(y_val, predicted_labels)
print(f'Accuracy: {accuracy:.2f}')

120/120 ━━━━━━━━━━━━━━━━━━━━ 391s 3s/step
Accuracy: 0.89


## using the loaded model to predict gender

In [4]:
def gen_detect(image_path):
    preprocessed = preprocess_image(image_path)
    if preprocessed is None:
        return "No face detected"
    preprocessed = np.expand_dims(preprocessed, axis=0)
    output = loaded_model(preprocessed)
    if(output<0.5):
        return "Male"
    else:
        return "Female"

In [6]:
gen_detect('/Users/saatvik/Desktop/secytasks/CVtask/with_mask/with_mask_3.jpg')

'Female'